# Scoring with Managed Batch Endpoints

In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient


credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<Subscription ID>"
    resource_group = "<ResourceGroup Name>"
    workspace = "<<WorkspaceName>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
print(ml_client)

We could not find config.json in: . or in its parent directories. 
MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f1ea0441730>,
         subscription_id=25758354-2fe7-426a-be0e-1ad20058340d,
         resource_group_name=azureml-live,
         workspace_name=ftaliveazureml)


In [2]:
#Get the compute target
cluster_name = 'cpu-cluster'
compute_target = ml_client.compute.get(name=cluster_name)

In [3]:
#Batch Inferencing
# import required libraries
from azure.ai.ml import Input
from azure.ai.ml.entities import (
    AmlCompute,
    BatchEndpoint,
    BatchDeployment,
    Model,
    Environment,
    BatchRetrySettings,
)
from azure.ai.ml.entities._assets import Dataset
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import BatchDeploymentOutputAction


In [4]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

batch_endpoint_name = "test-batch-endpoint-" + datetime.datetime.now().strftime(
    "%Y%m%d%H%M"
)

# create a batch endpoint
batch_endpoint = BatchEndpoint(
    name=batch_endpoint_name,
    description="Diabetes model sample Batch Endpoint",
    tags={"model": "diabetes trained"},
)

In [5]:
ml_client.batch_endpoints.begin_create_or_update(batch_endpoint)

BatchEndpoint({'scoring_uri': 'https://test-batch-endpoint-202206220906.southeastasia.inference.ml.azure.com/jobs', 'swagger_uri': None, 'provisioning_state': 'Succeeded', 'name': 'test-batch-endpoint-202206220906', 'description': 'Diabetes model sample Batch Endpoint', 'tags': {'model': 'diabetes trained'}, 'properties': {'BatchEndpointCreationApiVersion': '2022-05-01', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/providers/Microsoft.MachineLearningServices/locations/southeastasia/mfeOperationsStatus/be:b417752f-200a-46d5-b47e-800a43295269:099b9dd8-01b0-4991-8c73-5b955cbc4bd0?api-version=2022-05-01'}, 'id': '/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/resourceGroups/azureml-live/providers/Microsoft.MachineLearningServices/workspaces/ftaliveazureml/batchEndpoints/test-batch-endpoint-202206220906', 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f1e83123ee0>, 'au

In [6]:
# create a batch deployment
model = ml_client.models.get("Diabetest-LGBM-Model", version=1)

deployment = BatchDeployment(
    name="diabtest-deployment",
    description="this is a sample deployment",
    endpoint_name=batch_endpoint_name,
    model=model,
    environment="AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu@latest",
    compute=compute_target.name,
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
    logging_level="info",
)

In [7]:
ml_client.batch_deployments.begin_create_or_update(deployment)

BatchDeployment({'endpoint_name': 'test-batch-endpoint-202206220906', 'type': None, 'name': 'diabtest-deployment', 'description': 'this is a sample deployment', 'tags': {}, 'properties': {}, 'id': '/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/resourceGroups/azureml-live/providers/Microsoft.MachineLearningServices/workspaces/ftaliveazureml/batchEndpoints/test-batch-endpoint-202206220906/deployments/diabtest-deployment', 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f1e83150940>, 'model': '/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/resourceGroups/azureml-live/providers/Microsoft.MachineLearningServices/workspaces/ftaliveazureml/models/Diabetest-LGBM-Model/versions/1', 'code_configuration': None, 'environment': '/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/resourceGroups/azureml-live/providers/Microsoft.MachineLearningServices/workspaces/ftaliveazureml/environments/AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu/ve

In [8]:
# create a dataset form the folderpath
batch_input = ml_client.data.get(name="diabetes-dataset-uri-file", version="3")

file_input = Input(type="uri_file",
            path="https://azuremlexamples.blob.core.windows.net/datasets/diabetes.csv")

# invoke the endpoint for batch scoring job
job = ml_client.batch_endpoints.invoke(
    endpoint_name="test-batch-endpoint-202206210804",
    input=file_input,  
    deployment_name= "diabtest-deployment",# name is required as default deployment is not set
    params_override=[{"mini_batch_size": "20"}, {"compute.instance_count": "2"}],
)

ClientAuthenticationError: Operation returned an invalid status 'None'